# Nomogram Construction

Import Data + Models + Libraries/Packages

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.feature_lists import get_feature_lists
from simpleNomo import nomogram
import pandas as pd
import joblib
from src.config import SEED
## Data
y_train = (pd.read_excel('../data/raw/split/Raw_y_train.xlsx'))['ORN']
nomo_X_train = pd.read_parquet('../data/processed/nomo_train_transformed.parquet')

## Model
nomo_clf = joblib.load('../models/NLR.joblib')

Initialize feature lists

In [ ]:
##Imported func from src
feature_lists = get_feature_lists()
binary_cols = feature_lists["binary_cols"]
numerical_cols = feature_lists["numerical_cols"]
nominal_cols = feature_lists["nominal_cols"]
ordinal_cols = feature_lists["ordinal_cols"]

In [ ]:
##Get coef and intercept from log model
coefs = nomo_clf.coef_[0]
intercept = nomo_clf.intercept_[0]
##Create features and coef columns

feature_names = nomo_X_train.columns.to_list()
features = ['intercept'] + ['threshold'] + feature_names
coefs = [intercept] + [0.502] + list(coefs)

##Get continuous, nominal, and ordinal columns
df_size = nomo_X_train.shape[1]
cat_cols = [0] * df_size ## 0 = numerical
for i in range(0,df_size):
    col = nomo_X_train.columns[i]
    counts = nomo_X_train[col].value_counts()
    if col in nominal_cols or col in binary_cols: #1 = #Nominal
        cat_cols[i] = 1
    elif col in ordinal_cols: ##2 = Ordinal
        cat_cols[i] = 2

##Get mins, maxs, coefs, types, positions columns
mins = [''] + [''] +[nomo_X_train[feature].min() for feature in feature_names]
maxs = [''] + [''] +[nomo_X_train[feature].max() for feature in feature_names]
coefs = [1e-10 if i == 0 else i for i in coefs] #Make 0 entries a very small number
types = [''] +  [''] + ['numerical' if i == 0 else 'nominal' if i ==1 else 'ordinal' for i in cat_cols]
positions = [''] * (df_size+2)

##Create table
nomo_table = pd.DataFrame({
    'feature': features,
    'coef': coefs,
    'min': mins,
    'max': maxs,
    'type': types,
    'position': positions
})

##Export and import table
path_to_nomo_table = '../results/tables/Nomo_Table.xlsx'
nomo_table.to_excel(path_to_nomo_table, index = False)

##Create nomogram
results = nomogram(path = path_to_nomo_table, result_title = 'ORN Risk', 
                   fig_width=30, single_height=.7, dpi = 500,
                   ax_para={'linewidth': 2, 'c': 'black', 'linestyle': '--'},
                   xtick_para = {'fontsize': 13}
                   )
results.savefig('../results/figures/Nomogram.pdf', bbox_inches='tight', dpi=500)